In [90]:
from traitlets.config.manager import BaseJSONConfigManager
cm = BaseJSONConfigManager()
cm.update('livereveal', {
              'theme': 'sky',
              'transition': 'zoom',
              'start_slideshow_at': 'selected',
            'scroll': True
})

{'scroll': True,
 'start_slideshow_at': 'selected',
 'theme': 'sky',
 'transition': 'zoom'}

# Занятие 13. 

## Динамическое программирование


## План занятия

- квадратичные сортировки (повторение)
- задачи нахождения количества способов
    - метод перебора
    - комбинаторный метод
    - динамическое программирование

## Квадратичные сортировки (повторение)

- сортировка выбором
- сортировка пузырьком


### Сортировка выбором

#### Алгоритм:
- находим индекс минимального элемента
- меняем местами его с первым
- продолжаем итерации для списка длины $n - 1$

#### Сложность алгоритма:
- Время $\mathcal{O}(n^2)$
- Дополнительная память $\mathcal{O}(1)$

In [6]:
def select_sort(b):
    # работает как sorted
    x = b[:]
    n = len(x)
    for i in range(n-1):
        imin = i
        for j in range(i, n):
            if x[imin] > x[j]:
                imin = j
        x[i], x[imin] = x[imin], x[i]
    return x

In [7]:
x = [4, 10, -2, 3, 1, 0]
select_sort(x)

[-2, 0, 1, 3, 4, 10]

### Сортировка пузырьком

#### Алгоритм:
- **Правило**: левый элемент должен быть меньше правого
- идем слева направо, проверяя каждую пару на соответствие правилу
- при несоответствии меняем элементы местами
- продолжаем итерации для списка длины $n - 1$
- **Оптимизация**: на каждой итерации считаем количество элементов, которые поменялись. Если оно равняется нулю, то список уже отсортирован.

#### Сложность алгоритма:
- Время $\mathcal{O}(n^2)$
- Дополнительная память $\mathcal{O}(1)$

In [10]:
def bubble_sort(b):
    x = b[:]
    n = len(x)
        
    for i in range(n):
        for j in range(n-i-1):
            if x[j] > x[j+1]:
                x[j], x[j+1] = x[j+1], x[j]
#         print(f'iter={i}, {x}')
    return x

In [11]:
x = [4, 10, -2, 3, 1, 0]
bubble_sort(x)

[-2, 0, 1, 3, 4, 10]

### Сортировка вставками

#### Алгоритм:
- **Правило**: левый элемент должен быть меньше правого
- На $i$ итерации рассматриваем $i+1$-й элемент и меняем его с соседом слева до тех пор, пока не между ними не будет выполнено правило
- Проводим $n-1$ итерацию

#### Сложность алгоритма:
- Время $\mathcal{O}(n^2)$
- Дополнительная память $\mathcal{O}(1)$

In [14]:
def insert_sort(b):
    x = b[:]
    n = len(x)
    
    for i in range(1, n): ## максимальное значение i = n-1
        j = i - 1
        while j >= 0 and x[j] > x[j + 1] : ## x[-1]? x[-n]?
            x[j], x[j + 1] = x[j+ 1], x[j]
            j -= 1
    return x

In [15]:
x = [4, 10, -2, 3, 1, 0]
insert_sort(x)

[-2, 0, 1, 3, 4, 10]

### Время работы стандартной и квадратичной сортировок

In [19]:
from time import time
from random import randint

x = [randint(1, 10000) for i in range(10000)]
begin = time()
insert_sort(x)
print(f'Insert sort time: {time() - begin}')

begin = time()
sorted(x)
print(f'Python sorted time: {time() - begin}')


Insert sort time: 5.706234693527222
Python sorted time: 0.0025458335876464844


Сортировку можно применять не только для чисел, но и строк. Тогда они по умолчанию будут отсортированы в лексикографическом порядке

In [26]:
strs = ['dd', 'abc', 'ar', 'bad', 'sdsdcsd', 'a', 'drg']
insert_sort(strs)

['a', 'abc', 'ar', 'bad', 'dd', 'drg', 'sdsdcsd']

Если мы хотим отсортировать по длине строк, то необходимо только изменить правило 

In [27]:
def insert_sort_length(b):
    x = b[:]
    n = len(x)
    
    for i in range(1, n): ## максимальное значение i = n-1
        j = i - 1
        while j >= 0 and len(x[j]) > len(x[j + 1]) : ## x[-1]? x[-n]?
            x[j], x[j + 1] = x[j+ 1], x[j]
            j -= 1
    return x

insert_sort_length(strs)

['a', 'dd', 'ar', 'abc', 'bad', 'drg', 'sdsdcsd']

Плохо каждый раз писать новую функцию для изменения параметра сортировки, поэтому решением будет добавить параметр **key**, вместо которого будем передавать функцию, по которой мы хотим сортировать. По умолчанию будет обычный (лексикографический) порядок.

In [32]:
def insert_sort(b, key=lambda x: x):
    x = b[:]
    n = len(x)
    
    for i in range(1, n): ## максимальное значение i = n-1
        j = i - 1
        while j >= 0 and key(x[j]) > key(x[j + 1]) : ## x[-1]? x[-n]?
            x[j], x[j + 1] = x[j+ 1], x[j]
            j -= 1
    return x

print(f'Sort by length: {insert_sort(strs, key=len)}')
print(f'Sort by alphabet: {insert_sort(strs)}')


Sort by length: ['a', 'dd', 'ar', 'abc', 'bad', 'drg', 'sdsdcsd']
Sort by alphabet: ['a', 'abc', 'ar', 'bad', 'dd', 'drg', 'sdsdcsd']


### Задачи на нахождение количества способов

- Комбинаторные формулы $\mathcal{O}(1)$
- Динамическое программирование $\mathcal{O}(n)$
- Перебор $\mathcal{O}(2^n)$

### Задачи:

1) Сколько существует последовательностей, состоящих из $0$ и $1$ длины $n$? 

- Ответ можно получить просто перебрав все последовательности. 
Сложность $\mathcal{O}(2^n)$

- Комбинаторная формула дает ответ $2^n$. 
Сложность $\mathcal{O}(1)$

### Задачи:

2) Перечислить все последовательности, состоящие из $0$ и $1$ длины $n$? 

- Решаем с помощью рекурсивного перебора за $\mathcal{O}(2^n)$

In [39]:
def seq_n(n, s = ''):
    if n == 0:
        print(s)
        return
    seq_n(n-1, s + '0')
    seq_n(n-1, s + '1')

In [40]:
seq_n(3)

000
001
010
011
100
101
110
111


### Задачи:

3) Посчитать все последовательности, состоящие из $0$ и $1$ длины $n$, в которых ровно $k$ единиц.

- Комбинаторное ответ $C_n^k$. Сложность $\mathcal{O}(1)$

- Решаем с помощью рекурсивного перебора за $\mathcal{O}(2^n)$

In [65]:
def combk(n, k, s = ''):
    if n == 0:
        global count
        count += 1
        return
    
    if k > 0:
        combk(n-1, k - 1, s + '1')
    
    if n - k > 0: 
        combk(n-1, k, s + '0')

In [67]:
from math import factorial

count = 0
combk(100, 2)
print(f'Количество последовательность, посчитанных рекурсивно: {count}')

ans = factorial(100) // (factorial(2) * factorial(98))
print(f'Количество последовательность, посчитанных по формуле: {ans}')

Количество последовательность, посчитанных рекурсивно: 4950
Количество последовательность, посчитанных по формуле: 4950


### Задачи:

4) Посчитать все последовательности, состоящие из $0$ и $1$ длины $n$, в которых $2$ единицы не стоят рядом.

- Динамическое программирование! 


Пусть нам известны $f(n-1)$ - количество последовательностей, где 2 единицы не стоят рядом длины $n-1$, $f(n-2), ..., f(1)$. Выразим $f(n)$ через них. Есть 2 вида последовательностей
$$\text{начинающиеся с 0 --- } 0\underbrace{\cdots}_{n-1} \text{ их f(n-1)}$$
$$\text{начинающиеся с 1 --- } 10\underbrace{\cdots}_{n-2} \text{ их f(n-2)}$$

Следовательно:

$$ f(n) = f(n-1) + f(n-2) $$

Начальные условия:

$f(1) = 2, f(2) = 3$


In [74]:
def count_seq_2(n):
    f = [0]*(n+1)
    if n == 1:
        return 2
    if n == 2:
        return 3
    
    f[1] = 2
    f[2] = 3
    for i in range(3, n+1):
        f[i] = f[i-1] + f[i - 2]
    return f[-1]

print(count_seq_2(4))

8


4.2) Посчитать все последовательности, состоящие из $0$ и $1$ длины $n$, в которых $3$ единицы не стоят рядом.

- Аналогичные рассуждения приводят к формуле

$$ f(n) = f(n-1) + f(n-2) + f(n-3) $$

Начальные условия:

$f(1) = 2, f(2) = 4, f(3) = 7$

Количество начальных условий равно количеству значений необходимых для вычисления $f(n)$!!!

In [75]:
def count_seq_3(n):
    f = [0]*(n+1)
    if n == 1:
        return 2
    if n == 2:
        return 4
    if n == 3:
        return 7
    
    f[1] = 2
    f[2] = 3
    f[3] = 7
    for i in range(4, n+1):
        f[i] = f[i-1] + f[i - 2] + f[i - 3]
    return f[-1]

print(count_seq_3(5))

22


### Задачи о наилучшем способе:

1) Петя поднимается по ступенькам. Он может шагать по каждой, либо перепрыгивать через одну. Заданы стоимости прохода через каждую ступеньку. Необходимо найти минимальную стоимость прохода по лестнице.

- Пусть нам известны: 

$f(n-1)$ - минимальная стоимость прохода по лестнице длины $n-1, f(n-2), ..., f(1)$. 

$c[1], ..., c[n]$ -- стоимость каждой ступеньки. 

$$f(n) = \min{(f(n-1), f(n-2))} + c[n] $$

Начальные значения:

$f(1) = c[1]$

$f(2) = c[2]$

In [80]:
# +1 и +2 
def up_staircase(costs):
    costs.append(0)
    n = len(costs)
    f = [0]*(n+1)
    f[1] = costs[0]
    f[2] = costs[1]
    for i in range(3, n+1):
        f[i] = min(f[i-1], f[i -2]) + costs[i - 1]
    return f[-1]

costs = [4, 2, 5, 3, 10, 4, 2]
print(up_staircase(costs))

9


### Задачи о наилучшем способе:

Если добавить Пете возможность перешагивать через 2 ступеньки (+3), то формула примет вид:

 $$f(n) = \min{(f(n-1), f(n-2), f(n-3)} + c[n] $$ 

In [83]:
# +1, +2 и +3
def up_staircase_2(costs):
    n = len(costs)
    f = [0]*(n+1)
    f[1] = costs[0]
    f[2] = costs[1]
    f[3] = costs[2]
    for i in range(4, n+1):
        f[i] = min(f[i-1], f[i -2], f[i-3]) + costs[i - 1]
    return min(f[-1], f[-2], f[-3])

costs = [4, 2, 5, 3, 10, 4, 2]
print(up_staircase_2(costs))

7


In [84]:
new_costs = [2, 3, 1000, 1, 4] 
print(up_staircase(new_costs))
print(up_staircase_2(new_costs))

4
3


### Задачи о наилучшем способе:

2) Как выяснить по каким ступенькам необходимо пройти, чтобы заплатить минимальную стоимость?

- Решаем задачу по поиску минимальной стоимости пути. 
- Идем обратно: на ступеньку $n$ мы попали либо из $n-1$ либо $n-2$. На самом деле, где значение $f$ меньше, оттуда мы и попали. Запоминаем эту ступеньку и вычисляем, как мы попали в нее и т.д. до начальной

In [86]:
def up_staircase_1(costs):
    costs.append(0)
    n = len(costs)
    f = [0]*(n+1)
    f[1] = costs[0]
    f[2] = costs[1]
    for i in range(3, n+1):
        f[i] = min(f[i-1], f[i -2]) + costs[i - 1]
        
    cur_index = n
    path = []
    while cur_index > 0:
        path.append(cur_index)
        if f[cur_index] - costs[cur_index - 1] == f[cur_index - 1]:
            cur_index -= 1
        elif f[cur_index] - costs[cur_index - 1] == f[cur_index - 2]:
            cur_index -= 2
    
    return f[-1], path[::-1]

x = [2, 3, 1000, 1, 4] 
up_staircase_1(x)

In [12]:
from random import randint
costs = [randint(1, 20) for i in range(15)]
up_staircase_1(costs)

(67, [2, 4, 5, 7, 9, 10, 12, 14, 16])

In [89]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()